In [1]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

In [2]:
necoc = 500
generations = int(3000/necoc)
generations

6

In [3]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [4]:
df = pd.read_csv('letter-recognition.data',header=None)

train_df = df[:1600]
test_df = df[16000:16000+400]

X = train_df.ix[:,1:16].to_numpy() 
y = np.array([ord(l)-ord('A') for l in train_df.ix[:,0].to_numpy()])

testX = test_df.ix[:,1:16].to_numpy() 
testy = np.array([ord(l)-ord('A') for l in test_df.ix[:,0].to_numpy()])

In [5]:
def get_winner(class1):
    
    numbers = [*range(26)] # number 2 > number 1
    number_of_classification = len(numbers)
    #number_of_sampling = 100

    list_y = y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

        
    class2 = []

    class1_indexes = []
    class2_indexes = []

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("number of class1:{}".format(len(class1_label)))
    print("number of class2:{}".format(len(class2_label)))

    samplesize = 1600
    x_inputs =[tuple(c) for c in X[class1_indexes].tolist()] + [tuple(c) for c in X[class2_indexes].tolist()]
    x_outputs = [tuple([c]) for c in class1_label + class2_label]
    
    
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            outputs = []
            for xi in x_inputs:
                output = net.activate(xi)
                outputs.append(output)


            px_outputs = softmax(np.array(outputs).reshape(samplesize, 2), axis=1)
            # the index of maximum in each line
            pred_outputs = np.argmax(px_outputs, axis = 1)
            real_outputs = np.array(x_outputs).reshape(samplesize,)

            acc = np.sum(pred_outputs == real_outputs)/samplesize

            genome.fitness = acc

    def run(config_file):
        # Load configuration.
        config = neat.Config(
            neat.DefaultGenome,
            neat.DefaultReproduction,
            neat.DefaultSpeciesSet,
            neat.DefaultStagnation,
            config_file,
        )

        # Create the population, which is the top-level object for a NEAT run.
        p = neat.Population(config)

        # add a stdout reporter to show progress in the terminal
        reporter = neat.StdOutReporter(False)
        p.add_reporter(reporter)
        stats = neat.StatisticsReporter()
        p.add_reporter(stats)
        #checkpointer = neat.Checkpointer(100)
        #p.add_reporter(checkpointer)
        # Run for up to 300 generations.
        winner = p.run(eval_genomes, generations)

        return [stats, winner]
    
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward-letter2")
    [stats, winner] = run(config_path)
    
    return winner

In [6]:
import numpy as np
import random

number_codes = []
for i in range(2**(26-1), 2**26-1):
    if (i % 1000000) == 0:
            print(i)
    for j in bin(i).split('b')[1]:
        number_codes.append(int(j))
        


number_codes = np.array(number_codes).reshape(2**(26 - 1) - 1,26).T     

slice = random.sample(range(2**(26 - 1) - 1), necoc)

number_codes = number_codes[:,slice].tolist()

classes = []
for i in range(necoc):
    code_list = np.array(number_codes).T[i]
    classes.append(np.where(code_list==1)[0].tolist())

matrix_errors = []

for i in range(25):
    for j in range(i+1,26):
        matrix_errors.append(necoc - np.sum(np.array(number_codes[i]) == np.array(number_codes[j])))
min_error = np.min(matrix_errors)

print(min_error)

34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000
223


In [7]:
winner_list = []

for i,class1 in enumerate(classes):
    print("running:{}/30".format(i))
    winner = get_winner(class1)
    winner_list.append(winner)

running:0/30
number of class1:679
number of class2:921

 ****** Running generation 0 ****** 

Population's average fitness: 0.49893 stdev: 0.06849
Best fitness: 0.58875 - size: (2, 3) - species 9 - id 161
Average adjusted fitness: 0.089
Mean genetic distance 2.634, standard deviation 0.464
Population of 200 members in 14 species
Total extinctions: 0
Generation time: 2.721 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.53089 stdev: 0.06468
Best fitness: 0.59625 - size: (2, 2) - species 1 - id 301
Average adjusted fitness: 0.139
Mean genetic distance 2.763, standard deviation 0.515
Population of 199 members in 16 species
Total extinctions: 0
Generation time: 3.305 sec (3.013 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.53306 stdev: 0.06719
Best fitness: 0.59625 - size: (2, 2) - species 1 - id 301
Average adjusted fitness: 0.147
Mean genetic distance 2.822, standard deviation 0.532
Population of 200 members in 16 species
Total

Population's average fitness: 0.50290 stdev: 0.04121
Best fitness: 0.58375 - size: (2, 3) - species 1 - id 116
Average adjusted fitness: 0.085
Mean genetic distance 2.718, standard deviation 0.513
Population of 201 members in 13 species
Total extinctions: 0
Generation time: 2.519 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.51559 stdev: 0.04256
Best fitness: 0.60000 - size: (3, 5) - species 1 - id 365
Average adjusted fitness: 0.112
Mean genetic distance 2.736, standard deviation 0.556
Population of 201 members in 14 species
Total extinctions: 0
Generation time: 2.897 sec (2.708 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.51881 stdev: 0.04480
Best fitness: 0.60437 - size: (2, 4) - species 5 - id 443
Average adjusted fitness: 0.095
Mean genetic distance 2.781, standard deviation 0.568
Population of 201 members in 16 species
Total extinctions: 0
Generation time: 3.199 sec (2.871 average)

 ****** Running generation 3 *****


 ****** Running generation 0 ****** 

Population's average fitness: 0.49985 stdev: 0.01702
Best fitness: 0.55875 - size: (2, 3) - species 4 - id 199
Average adjusted fitness: 0.069
Mean genetic distance 2.685, standard deviation 0.466
Population of 199 members in 14 species
Total extinctions: 0
Generation time: 3.144 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.50439 stdev: 0.01719
Best fitness: 0.56937 - size: (3, 5) - species 6 - id 318
Average adjusted fitness: 0.042
Mean genetic distance 2.686, standard deviation 0.485
Population of 198 members in 15 species
Total extinctions: 0
Generation time: 3.569 sec (3.356 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.50675 stdev: 0.01984
Best fitness: 0.56937 - size: (3, 5) - species 11 - id 318
Average adjusted fitness: 0.062
Mean genetic distance 2.665, standard deviation 0.504
Population of 198 members in 17 species
Total extinctions: 0
Generation time: 3.254 sec (3.322 aver

Population's average fitness: 0.49306 stdev: 0.16405
Best fitness: 0.70188 - size: (2, 3) - species 3 - id 161
Average adjusted fitness: 0.190
Mean genetic distance 2.578, standard deviation 0.425
Population of 199 members in 10 species
Total extinctions: 0
Generation time: 2.524 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.56899 stdev: 0.16034
Best fitness: 0.70188 - size: (2, 3) - species 5 - id 161
Average adjusted fitness: 0.282
Mean genetic distance 2.640, standard deviation 0.496
Population of 200 members in 13 species
Total extinctions: 0
Generation time: 2.921 sec (2.722 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.60313 stdev: 0.14616
Best fitness: 0.70188 - size: (2, 3) - species 3 - id 161
Average adjusted fitness: 0.288
Mean genetic distance 2.623, standard deviation 0.509
Population of 200 members in 14 species
Total extinctions: 0
Generation time: 3.106 sec (2.850 average)

 ****** Running generation 3 *****

Population's average fitness: 0.49842 stdev: 0.03923
Best fitness: 0.58437 - size: (2, 3) - species 1 - id 38
Average adjusted fitness: 0.100
Mean genetic distance 2.591, standard deviation 0.451
Population of 201 members in 10 species
Total extinctions: 0
Generation time: 2.587 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.51177 stdev: 0.04072
Best fitness: 0.60813 - size: (3, 5) - species 8 - id 276
Average adjusted fitness: 0.096
Mean genetic distance 2.580, standard deviation 0.449
Population of 199 members in 12 species
Total extinctions: 0
Generation time: 2.904 sec (2.746 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.51268 stdev: 0.04221
Best fitness: 0.60813 - size: (3, 5) - species 8 - id 276
Average adjusted fitness: 0.086
Mean genetic distance 2.615, standard deviation 0.468
Population of 200 members in 12 species
Total extinctions: 0
Generation time: 3.066 sec (2.852 average)

 ****** Running generation 3 ******

...

Population's average fitness: 0.49502 stdev: 0.22095
Best fitness: 0.75125 - size: (2, 3) - species 2 - id 153
Average adjusted fitness: 0.263
Mean genetic distance 2.740, standard deviation 0.516
Population of 200 members in 13 species
Total extinctions: 0
Generation time: 2.411 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.60035 stdev: 0.20284
Best fitness: 0.75250 - size: (3, 4) - species 3 - id 310
Average adjusted fitness: 0.374
Mean genetic distance 2.818, standard deviation 0.561
Population of 203 members in 17 species
Total extinctions: 0
Generation time: 2.726 sec (2.569 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.58409 stdev: 0.20896
Best fitness: 0.75250 - size: (3, 4) - species 4 - id 310
Average adjusted fitness: 0.349
Mean genetic distance 2.839, standard deviation 0.583
Population of 198 members in 17 species
Total extinctions: 0
Generation time: 2.975 sec (2.704 average)

 ****** Running generation 3 *****

Population's average fitness: 0.49002 stdev: 0.12839
Best fitness: 0.65625 - size: (2, 3) - species 6 - id 168
Average adjusted fitness: 0.177
Mean genetic distance 2.714, standard deviation 0.526
Population of 200 members in 12 species
Total extinctions: 0
Generation time: 2.378 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.55217 stdev: 0.12277
Best fitness: 0.69937 - size: (2, 3) - species 10 - id 308
Average adjusted fitness: 0.215
Mean genetic distance 2.790, standard deviation 0.562
Population of 200 members in 15 species
Total extinctions: 0
Generation time: 2.731 sec (2.555 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.55585 stdev: 0.12287
Best fitness: 0.69937 - size: (2, 3) - species 10 - id 308
Average adjusted fitness: 0.218
Mean genetic distance 2.880, standard deviation 0.600
Population of 199 members in 18 species
Total extinctions: 0
Generation time: 2.863 sec (2.657 average)

 ****** Running generation 3 ***

Population's average fitness: 0.50660 stdev: 0.08414
Best fitness: 0.65063 - size: (2, 3) - species 7 - id 64
Average adjusted fitness: 0.135
Mean genetic distance 2.615, standard deviation 0.461
Population of 201 members in 11 species
Total extinctions: 0
Generation time: 2.371 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.54658 stdev: 0.07507
Best fitness: 0.65063 - size: (2, 3) - species 7 - id 64
Average adjusted fitness: 0.172
Mean genetic distance 2.737, standard deviation 0.491
Population of 201 members in 12 species
Total extinctions: 0
Generation time: 2.687 sec (2.529 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.55087 stdev: 0.07620
Best fitness: 0.66625 - size: (3, 4) - species 12 - id 494
Average adjusted fitness: 0.223
Mean genetic distance 2.730, standard deviation 0.545
Population of 201 members in 13 species
Total extinctions: 0
Generation time: 2.827 sec (2.628 average)

 ****** Running generation 3 ******

Population's average fitness: 0.50247 stdev: 0.03123
Best fitness: 0.60688 - size: (2, 3) - species 6 - id 192
Average adjusted fitness: 0.115
Mean genetic distance 2.749, standard deviation 0.510
Population of 199 members in 14 species
Total extinctions: 0
Generation time: 2.376 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.51771 stdev: 0.03779
Best fitness: 0.68563 - size: (3, 4) - species 7 - id 260
Average adjusted fitness: 0.097
Mean genetic distance 2.807, standard deviation 0.517
Population of 198 members in 18 species
Total extinctions: 0
Generation time: 2.664 sec (2.520 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.52090 stdev: 0.04301
Best fitness: 0.68563 - size: (3, 4) - species 7 - id 260
Average adjusted fitness: 0.117
Mean genetic distance 2.845, standard deviation 0.568
Population of 198 members in 20 species
Total extinctions: 0
Generation time: 3.011 sec (2.683 average)

 ****** Running generation 3 *****

Population's average fitness: 0.49396 stdev: 0.15027
Best fitness: 0.68688 - size: (2, 3) - species 3 - id 154
Average adjusted fitness: 0.193
Mean genetic distance 2.613, standard deviation 0.443
Population of 199 members in 11 species
Total extinctions: 0
Generation time: 2.350 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.59014 stdev: 0.12679
Best fitness: 0.68688 - size: (2, 3) - species 2 - id 154
Average adjusted fitness: 0.267
Mean genetic distance 2.689, standard deviation 0.515
Population of 200 members in 13 species
Total extinctions: 0
Generation time: 2.668 sec (2.509 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.57890 stdev: 0.13335
Best fitness: 0.68688 - size: (2, 3) - species 4 - id 154
Average adjusted fitness: 0.252
Mean genetic distance 2.707, standard deviation 0.516
Population of 200 members in 14 species
Total extinctions: 0
Generation time: 2.973 sec (2.664 average)

 ****** Running generation 3 *****

In [8]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-letter2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = testX[j]
    test_y = testy[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

from random import randint
pred_value = []
error_list = []

for j in range(400):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(necoc - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )
list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
np.sum(list_P == testy)/400

0.4975

In [9]:
def get_pred_train(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = X[j]
    test_y = y[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

pred_value = []
error_list = []

for j in range(1600):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_train(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(26 - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )

list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
print("Trainset acc:{}".format(np.sum(list_P == y)/1600))
winner_fitness = []
for winner in winner_list:
    #print(winner.fitness)
    winner_fitness.append(winner.fitness)
print("Avg Base Learner:{}".format(np.mean(winner_fitness)))

Trainset acc:0.58
Avg Base Learner:0.65673


In [10]:
# program to check if there is exist a path between two vertices 
# of a graph 

from collections import defaultdict 

#This class represents a directed graph using adjacency list representation 
class Graph: 

	def __init__(self,vertices): 
		self.V= vertices #No. of vertices 
		self.graph = defaultdict(list) # default dictionary to store graph 

	# function to add an edge to graph 
	def addEdge(self,u,v): 
		self.graph[u].append(v) 
	
	# Use BFS to check path between s and d 
	def isReachable(self, s, d): 
		# Mark all the vertices as not visited 
		visited =[False]*(self.V) 

		# Create a queue for BFS 
		queue=[] 

		# Mark the source node as visited and enqueue it 
		queue.append(s) 
		visited[s] = True

		while queue: 

			#Dequeue a vertex from queue 
			n = queue.pop(0) 
			
			# If this adjacent node is the destination node, 
			# then return true 
			if n == d: 
				return True

			# Else, continue to do BFS 
			for i in self.graph[n]: 
				if visited[i] == False: 
					queue.append(i) 
					visited[i] = True
		# If BFS is complete without visited d 
		return False
    
    
def findAllPath(graph,start,end,path=[]):
    path = path +[start]
    if start == end:
        return [path]
 
    paths = [] #存储所有路径    
    for node in graph[start]:
        if node not in path:
            newpaths = findAllPath(graph,node,end,path) 
            for newpath in newpaths:
                paths.append(newpath)
    return paths

numbers = [*range(2)]
number_of_classification = len(numbers)

list_nodes_number = []
list_connection_number = []

for winner in winner_list:
    used_nodes = list(winner.nodes.keys())
    # create a dict for mapping 
    l1 = numbers + config.genome_config.input_keys + used_nodes[number_of_classification:]
    l2 = range(len(l1))
    dict_nodes = dict(zip(l1, l2))

    ### 将节点加入图
    g = Graph(len(l1))# inputs + outputs + used  
    # add connections in the graph
    for cg in winner.connections.values():
        if cg.enabled:
            g.addEdge(dict_nodes[cg.key[0]],dict_nodes[cg.key[1]])

    v = numbers
    list_connections = []
    for vi in v:
        for u in range(number_of_classification, len(config.genome_config.input_keys) + number_of_classification):
            list_connections.append(g.isReachable(u, vi))

    nodes_inputs = range(number_of_classification, number_of_classification+ 64)
    nodes_outputs = range(number_of_classification)
    nodes_mid = range(number_of_classification + 64, len(dict_nodes))

    # 找到所有输入和输出连接的路
    all_path = []
    for u in nodes_inputs:
        for v in nodes_outputs:
            path = findAllPath(g.graph, u, v)
            if path:
                all_path = all_path + path

    # 得到最长路，以及每个节点在哪个层级
    max_length = max([len(x) for x in all_path])
    nodes_tuples_list = []
    for path in all_path:
        #print(path)
        for node in path:
            nodes_tuples_list.append([node, path.index(node)])

    # 确定节点的层级（消除重复）       
    nodes_tuples_fixed_list = []
    for index in range(max_length):
        for nodes in nodes_tuples_list:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                if nodes not in nodes_tuples_fixed_list:
                    nodes_tuples_fixed_list.append(nodes)


    # 所有有连接的节点
    all_used_nodes = []
    for node in nodes_tuples_fixed_list:
        if node[0] not in all_used_nodes:
            all_used_nodes.append(node[0])

    # 确定节点的层级（最终确定）
    nodes_tuples_fixed_list_final = []

    for node in all_used_nodes:
        if node in nodes_inputs:
            nodes_tuples_fixed_list_final.append([node, 0])
        elif node in nodes_outputs:
            nodes_tuples_fixed_list_final.append([node, max_length - 1])
        else:
            list_node = []
            for nodes in nodes_tuples_fixed_list:
                if node == nodes[0]:
                    list_node.append(nodes)
            layer = 0
            for ele in list_node:
                # 删除重复项，取最大
                if ele[1] > layer:
                    layer = ele[1]
            nodes_tuples_fixed_list_final.append([node,layer])


    # 每个层级的节点合数
    length_of_layers = []
    number_of_nodes = 0
    for index in range(max_length):
        for nodes in nodes_tuples_fixed_list_final:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                number_of_nodes += 1
        length_of_layers.append(number_of_nodes)

    length_of_each_layer = []
    length_of_each_layer.append(length_of_layers[0])
    for i in range(1, max_length):
        length_of_each_layer.append(length_of_layers[i] - length_of_layers[i-1])

    # 输出每个层级的节点个数
    #print("length of each layers:", length_of_each_layer)

    # 所有端到端的路
    all_path_side2side = []
    for path in all_path:
        if len(path) == 2:
            all_path_side2side.append(path)
        else:
            for i in range(len(path)-1):
                all_path_side2side.append([path[i],path[i+1]])

    # 定义节点到绘图的字典
    dict_nodes_graph = dict()
    count_number_layer = [0] * max_length
    for nodes in nodes_tuples_fixed_list_final:
        node, layer = nodes
        dict_nodes_graph[node] = [count_number_layer[layer], layer]
        count_number_layer[layer] += 1
    list_nodes_number.append(np.sum(length_of_each_layer))
    list_connection_number.append(len(all_path_side2side))
    #print("Number of nodes:{} Number of connections:{}".format(np.sum(length_of_each_layer),len(all_path_side2side)))
print("Total nodes:{} Total connections::{}".format(np.sum(list_nodes_number), np.sum(list_connection_number)))

Total nodes:3588 Total connections::4367


In [12]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-letter2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = testX[j]
    test_y = testy[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    
    # the index of maximum in each line
    return px_outputs

from random import randint
pred_value = []
error_list = []

for j in range(400):
    pred = []
    for i in range(necoc):
        pred_outputs = get_pred_real(i, j)
        pred.append(pred_outputs)

    error = []
    for i in range(26):
        loss = 0
        for ii in range(len(pred)):
            #print(number_codes[0][ii], pred[ii][0])
            #print (pred[ii][0][number_codes[0][ii]])
            loss += (1 - pred[ii][0][number_codes[i][ii]])**2
        error.append(loss)

    pred_value.append(np.where(error==np.min(error)))

list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
np.sum(list_P == testy)/400

0.465